**Demo for `teneva.core.als`**

---

This module contains the function `als` which computes the TT-approximation for the tensor by TT-ALS algorithm, using given random samples.

## Loading and importing modules

In [1]:
import numpy as np
import teneva
from time import perf_counter as tpc
np.random.seed(42)

## Function `als`

Build TT-tensor by TT-ALS from the given random tensor samples.

In [2]:
d         = 5                           # Dimension of the function
a         = [-5., -4., -3., -2., -1.]   # Lower bounds for spatial grid
b         = [+6., +3., +3., +1., +2.]   # Upper bounds for spatial grid
n         = [ 20,  18,  16,  14,  12]   # Shape of the tensor

In [3]:
m         = 1.E+4                       # Number of calls to target function
nswp      = 50                          # Sweep number for ALS iterations
r         = 3                           # TT-rank of the initial random tensor

We set the target function (the function takes as input a set of tensor multi-indices I of the shape [samples, dimension], which are transformed into points X of a uniform spatial grid using the function "ind_to_poi"):

In [4]:
from scipy.optimize import rosen
def func(I): 
    X = teneva.ind_to_poi(I, a, b, n)
    return rosen(X.T)

In [5]:
def func(I):
    """Schaffer function."""
    X = teneva.ind_to_poi(I, a, b, n)
    Z = X[:, :-1]**2 + X[:, 1:]**2
    Y = 0.5 + (np.sin(np.sqrt(Z))**2 - 0.5) / (1. + 0.001 * Z)**2
    return np.sum(Y, axis=1)

We prepare train data from the LHS random distribution:

In [6]:
I_trn = teneva.sample_lhs(n, m) 
Y_trn = func(I_trn)

We prepare test data from as a random tensor multi-indices:

In [7]:
# Test data:

# Number of test points:
m_tst = int(1.E+4)

# Random multi-indices for the test points:
I_tst = np.vstack([np.random.choice(k, m_tst) for k in n]).T

# Function values for the test points:
Y_tst = func(I_tst)

We build the TT-tensor, which approximates the target function (we generate random initial r-rank approximation in the TT-format using the function "rand" and then compute the resulting TT-tensor by TT-ALS):

In [8]:
t = tpc()
Y = teneva.rand(n, r)
Y = teneva.als(I_trn, Y_trn, Y, nswp)
t = tpc() - t

print(f'Build time     : {t:-10.2f}')

Build time     :       2.72


And now we can check the result:

In [9]:
# Fast getter for TT-tensor values:
get = teneva.getter(Y)                     

# Compute approximation in train points:
Z = np.array([get(i) for i in I_trn])

# Accuracy of the result for train points:
e_trn = np.linalg.norm(Z - Y_trn)          
e_trn /= np.linalg.norm(Y_trn)

# Compute approximation in test points:
Z = np.array([get(i) for i in I_tst])

# Accuracy of the result for test points:
e_tst = np.linalg.norm(Z - Y_tst)          
e_tst /= np.linalg.norm(Y_tst)

print(f'Error on train : {e_trn:-10.2e}')
print(f'Error on test  : {e_tst:-10.2e}')

Error on train :   9.18e-01
Error on test  :   1.18e+00


We can also set a validation data set and specify as a stop criterion the accuracy of the TT-approximation on this data (and we can also present the logs):

In [10]:
# Validation data:

# Number of validation points:
m_vld = int(1.E+3)

# Random multi-indices for the validation points:
I_vld = np.vstack([np.random.choice(k, m_vld) for k in n]).T

# Function values for the validation points:
Y_vld = func(I_vld)

In [11]:
t = tpc()
Y = teneva.rand(n, r)
Y = teneva.als(I_trn, Y_trn, Y, nswp, I_vld=I_vld, Y_vld=Y_vld, e_vld=1.E-5, log=True)
t = tpc() - t

print(f'Build time     : {t:-10.2f}')

# pre | time:      0.585 | rank:   3.0 | err: 3.7e+00 | 
#   1 | time:      1.126 | rank:   3.0 | err: 1.0e+00 | eps: 1.0e+00 | 
#   2 | time:      1.730 | rank:   3.0 | err: 1.0e+00 | eps: 1.0e+00 | 
#   3 | time:      2.283 | rank:   3.0 | err: 2.4e-01 | eps: 2.0e+00 | 
#   4 | time:      2.777 | rank:   3.0 | err: 8.8e-02 | eps: 3.5e-01 | 
#   5 | time:      3.305 | rank:   3.0 | err: 6.0e-02 | eps: 6.2e-02 | 
#   6 | time:      3.845 | rank:   3.0 | err: 5.4e-02 | eps: 3.1e-02 | 
#   7 | time:      4.401 | rank:   3.0 | err: 5.4e-02 | eps: 1.1e-02 | 
#   8 | time:      4.872 | rank:   3.0 | err: 5.3e-02 | eps: 3.9e-03 | 
#   9 | time:      5.395 | rank:   3.0 | err: 5.3e-02 | eps: 2.0e-03 | 
#  10 | time:      5.946 | rank:   3.0 | err: 5.3e-02 | eps: 6.9e-04 | 
#  11 | time:      6.423 | rank:   3.0 | err: 5.3e-02 | eps: 3.5e-04 | 
#  12 | time:      6.946 | rank:   3.0 | err: 5.3e-02 | eps: 1.2e-04 | 
#  13 | time:      7.493 | rank:   3.0 | err: 5.3e-02 | eps: 6.1e-05 | 
#  14 |

We can use helper functions to present the resulting accuracy:

In [13]:
print(f'Error on train : {teneva.accuracy_on_data(Y, I_trn, Y_trn):-10.2e}')
print(f'Error on valid.: {teneva.accuracy_on_data(Y, I_vld, Y_vld):-10.2e}')
print(f'Error on test  : {teneva.accuracy_on_data(Y, I_tst, Y_tst):-10.2e}')

Error on train :   5.20e-02
Error on valid.:   5.34e-02
Error on test  :   5.38e-02


We may also set the value of relative rate of solution change to stop the iterations:

In [14]:
t = tpc()
Y = teneva.rand(n, r)
Y = teneva.als(I_trn, Y_trn, Y, e=1.E-6, I_vld=I_vld, Y_vld=Y_vld, log=True)
t = tpc() - t

print(f'Build time     : {t:-10.2f}')

# pre | time:      0.565 | rank:   3.0 | err: 3.1e+00 | 
#   1 | time:      1.043 | rank:   3.0 | err: 1.0e+00 | eps: 1.0e+00 | 
#   2 | time:      1.615 | rank:   3.0 | err: 1.0e+00 | eps: 7.1e-01 | 
#   3 | time:      2.204 | rank:   3.0 | err: 1.0e+00 | eps: 3.5e-01 | 
#   4 | time:      2.687 | rank:   3.0 | err: 8.9e-01 | eps: 1.4e+00 | 
#   5 | time:      3.220 | rank:   3.0 | err: 1.8e-01 | eps: 1.0e+00 | 
#   6 | time:      3.752 | rank:   3.0 | err: 6.5e-02 | eps: 1.8e-01 | 
#   7 | time:      4.318 | rank:   3.0 | err: 5.5e-02 | eps: 4.4e-02 | 
#   8 | time:      4.793 | rank:   3.0 | err: 5.4e-02 | eps: 1.1e-02 | 
#   9 | time:      5.322 | rank:   3.0 | err: 5.3e-02 | eps: 7.8e-03 | 
#  10 | time:      5.875 | rank:   3.0 | err: 5.3e-02 | eps: 3.9e-03 | 
#  11 | time:      6.350 | rank:   3.0 | err: 5.3e-02 | eps: 1.4e-03 | 
#  12 | time:      6.877 | rank:   3.0 | err: 5.3e-02 | eps: 6.9e-04 | 
#  13 | time:      7.439 | rank:   3.0 | err: 5.3e-02 | eps: 2.4e-04 | 
#  14 |

In [15]:
print(f'Error on train : {teneva.accuracy_on_data(Y, I_trn, Y_trn):-10.2e}')
print(f'Error on valid.: {teneva.accuracy_on_data(Y, I_vld, Y_vld):-10.2e}')
print(f'Error on test  : {teneva.accuracy_on_data(Y, I_tst, Y_tst):-10.2e}')

Error on train :   5.20e-02
Error on valid.:   5.34e-02
Error on test  :   5.38e-02


Note that we can use TT-ANOVA as an initial approximation (it will lead to faster convergence):

In [16]:
t = tpc()
Y = teneva.anova(I_trn, Y_trn, r)
Y = teneva.als(I_trn, Y_trn, Y, nswp, I_vld=I_vld, Y_vld=Y_vld, e_vld=1.E-5, log=True)
t = tpc() - t

print(f'Build time     : {t:-10.2f}')

# pre | time:      0.584 | rank:   3.0 | err: 2.1e-01 | 
#   1 | time:      1.073 | rank:   3.0 | err: 9.8e-02 | eps: 1.8e-01 | 
#   2 | time:      1.611 | rank:   3.0 | err: 9.0e-02 | eps: 3.1e-02 | 
#   3 | time:      2.174 | rank:   3.0 | err: 9.2e-02 | eps: 1.6e-02 | 
#   4 | time:      2.643 | rank:   3.0 | err: 9.2e-02 | eps: 7.8e-03 | 
#   5 | time:      3.170 | rank:   3.0 | err: 9.2e-02 | eps: 7.8e-03 | 
#   6 | time:      3.711 | rank:   3.0 | err: 8.9e-02 | eps: 5.5e-03 | 
#   7 | time:      4.271 | rank:   3.0 | err: 8.6e-02 | eps: 1.6e-02 | 
#   8 | time:      4.747 | rank:   3.0 | err: 8.2e-02 | eps: 5.5e-03 | 
#   9 | time:      5.281 | rank:   3.0 | err: 7.8e-02 | eps: 7.8e-03 | 
#  10 | time:      5.837 | rank:   3.0 | err: 7.4e-02 | eps: 1.1e-02 | 
#  11 | time:      6.315 | rank:   3.0 | err: 7.1e-02 | eps: 7.8e-03 | 
#  12 | time:      6.846 | rank:   3.0 | err: 6.9e-02 | eps: 3.9e-03 | 
#  13 | time:      7.387 | rank:   3.0 | err: 6.7e-02 | eps: 3.9e-03 | 
#  14 |

In [17]:
print(f'Error on train : {teneva.accuracy_on_data(Y, I_trn, Y_trn):-10.2e}')
print(f'Error on valid.: {teneva.accuracy_on_data(Y, I_vld, Y_vld):-10.2e}')
print(f'Error on test  : {teneva.accuracy_on_data(Y, I_tst, Y_tst):-10.2e}')

Error on train :   5.63e-02
Error on valid.:   5.84e-02
Error on test  :   5.87e-02


## Function `als2`

Build TT-tensor by TT-ALS from the given random tensor samples. This is the alternative realization of the algorithm. The version from "als" function in many cases works better and much faster.

In [16]:
d         = 5                           # Dimension of the function
a         = [-5., -4., -3., -2., -1.]   # Lower bounds for spatial grid
b         = [+6., +3., +3., +1., +2.]   # Upper bounds for spatial grid
n         = [ 20,  18,  16,  14,  12]   # Shape of the tensor

In [17]:
m         = 1.E+4                       # Number of calls to target function
nswp      = 50                          # Sweep number for ALS iterations
r         = 3                           # TT-rank of the initial random tensor

We set the target function (the function takes as input a set of tensor multi-indices I of the shape [samples, dimension], which are transformed into points X of a uniform spatial grid using the function "ind_to_poi"):

In [18]:
from scipy.optimize import rosen
def func(I): 
    X = teneva.ind_to_poi(I, a, b, n)
    return rosen(X.T)

We prepare train data from the LHS random distribution:

In [19]:
I_trn = teneva.sample_lhs(n, m) 
Y_trn = func(I_trn)

We prepare test data from as a random tensor multi-indices:

In [20]:
# Test data:

# Number of test points:
m_tst = int(1.E+4)

# Random multi-indices for the test points:
I_tst = np.vstack([np.random.choice(k, m_tst) for k in n]).T

# Function values for the test points:
Y_tst = func(I_tst)

We build the TT-tensor, which approximates the target function (we generate random initial r-rank approximation in the TT-format using the function "rand" and then compute the resulting TT-tensor by TT-ALS):

In [21]:
t = tpc()
Y = teneva.rand(n, r)
Y = teneva.als2(I_trn, Y_trn, Y, nswp)
t = tpc() - t

print(f'Build time     : {t:-10.2f}')

Build time     :      87.32


And now we can check the result:

In [22]:
# Fast getter for TT-tensor values:
get = teneva.getter(Y)                     

# Compute approximation in train points:
Z = np.array([get(i) for i in I_trn])

# Accuracy of the result for train points:
e_trn = np.linalg.norm(Z - Y_trn)          
e_trn /= np.linalg.norm(Y_trn)

# Compute approximation in test points:
Z = np.array([get(i) for i in I_tst])

# Accuracy of the result for test points:
e_tst = np.linalg.norm(Z - Y_tst)          
e_tst /= np.linalg.norm(Y_tst)

print(f'Error on train : {e_trn:-10.2e}')
print(f'Error on test  : {e_tst:-10.2e}')

Error on train :   7.53e-01
Error on test  :   2.61e+00


---